In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
pip install tensorflow==1.6.0

In [0]:
pip install keras==2.1.5

In [0]:
import tensorflow
import keras
print(tensorflow.__version__) # 1.6.0
print(keras.__version__) # 2.1.5
tensorflow.keras.__version__ # 2.1.3-tf


In [0]:
import matplotlib.pyplot as plt
import numpy as np
import random
random.seed(30)

In [0]:
dir_path = '/content/drive/My Drive/CS 280/dataset-resized'

classes = ('cardboard','glass', 'metal', 'paper', 'plastic', 'trash')
num_classes = 6

In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         validation_split=0.1,
                         rescale=1./255,
                         shear_range = 0.1,
                         zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1)

val_datagen=ImageDataGenerator(rescale=1./255, validation_split=0.1)

# With augmentation
train_generator=train_datagen.flow_from_directory(dir_path,
                                                  target_size=(224,224),
                                                  batch_size=128,
                                                  class_mode='categorical',
                                                  subset='training',
                                                  seed = 30)

val_generator=val_datagen.flow_from_directory(dir_path,
                                              target_size=(224,224),
                                              batch_size=128,
                                              class_mode='categorical',
                                              subset='validation',
                                              seed = 30)

labels = (train_generator.class_indices)
print(labels)

labels = dict((v,k) for k,v in labels.items())
print(labels)

In [0]:
# val = [next(val_generator) for i in range(0,5)] 
# fig, ax = plt.subplots(1,5, figsize=(15, 6)) 
# print('Labels:', [item[1][0] for item in val])
# l = [ax[i].imshow(val[i][0][0]) for i in range(0,5)]

In [0]:
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer, Activation, BatchNormalization, AveragePooling2D
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras import optimizers
from tensorflow.python.keras import backend as K

K.set_learning_phase(1)
resnet_model = ResNet50(include_top=False, weights='imagenet', input_shape= (224,224,3), classes=6)

for layer in resnet_model.layers:
    layer.trainable = False
    
    if layer.name.startswith('bn'):
        layer.call(layer.input, training=False)
  
resnet_model.summary()


In [0]:
model = Sequential()
model.add(resnet_model)

# Classification layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.4))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.4))

# SVM
from keras.regularizers import l2
model.add(Dense(6, activation='linear', kernel_regularizer=tensorflow.keras.regularizers.l2()))
model.compile(loss='categorical_hinge',
              optimizer=tensorflow.keras.optimizers.SGD(lr=0.005, momentum=0.9),
              metrics=['acc'])


model.summary()

In [0]:
# Load model
model.load_weights('/content/drive/My Drive/CS 280/restnet50.h5')
model

In [0]:
from keras.callbacks import ModelCheckpoint

filepath='/content/drive/My Drive/CS 280/restnet50_SVM.h5'
checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1,  mode='max')
callbacks_list = [checkpoint1]

history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=96,
      validation_data=val_generator,
      validation_steps=val_generator.samples/val_generator.batch_size,
      verbose=1,
      workers=4,
      callbacks=callbacks_list)

Epoch 1/96
17/17 [===========================>..]17/17 [===========================>..] - ETA: 1:18 - loss: 0.4437 - acc: 0.8566
Epoch 00001: saving model to /content/drive/My Drive/CS 280/restnet50_SVM.h5
18/17 [==============================]18/17 [==============================] - 1693s 94s/step - loss: 0.4360 - acc: 0.8603 - val_loss: 0.5489 - val_acc: 0.8175

Epoch 2/96
17/17 [===========================>..]17/17 [===========================>..] - ETA: 1:16 - loss: 0.4008 - acc: 0.8727
Epoch 00002: saving model to /content/drive/My Drive/CS 280/restnet50_SVM.h5
18/17 [==============================]18/17 [==============================] - 1673s 93s/step - loss: 0.4083 - acc: 0.8698 - val_loss: 0.4783 - val_acc: 0.8492

Epoch 3/96
17/17 [===========================>..]17/17 [===========================>..] - ETA: 1:16 - loss: 0.3905 - acc: 0.8888
Epoch 00003: saving model to /content/drive/My Drive/CS 280/restnet50_SVM.h5
18/17 [==============================]18/17 [===============

In [0]:
model.save('/content/drive/My Drive/CS 280/restnet50_SVM.h5')